In [1]:
from bs4 import BeautifulSoup as soup
import requests
import pandas as pd
import os

## Get Link

In [3]:
url = 'https://www.innovestx.co.th/faqs'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
}
manual_links = set()
for i in range(1,5):
    link = f'https://www.innovestx.co.th/products/product-user-guide/detail/{i}'
    response = requests.get(link,headers=headers)
    page_html = response.content
    page_soup = soup(page_html, 'html.parser')

    blog_card_headers = page_soup.find_all('div', class_='blog_card_header')
    
    # Extract href from a tags within these divs
    for header in blog_card_headers:
        a_tag = header.find('a')
        if a_tag and a_tag.get('href'):
            href = a_tag['href']
            # Optionally prepend the base URL if needed
            if not href.startswith('http'):
                href = 'https://www.innovestx.co.th' + href
            print(href)
            manual_links.add(href)

manual_links = list(manual_links)


https://www.innovestx.co.th/products/product-user-guide/detail/manual/webtrade-username
https://www.innovestx.co.th/products/product-user-guide/detail/manual/webtrade-email
https://www.innovestx.co.th/products/product-user-guide/detail/manual/webtrade-unlock
https://www.innovestx.co.th/products/product-user-guide/detail/manual/webtrade-overall
https://www.innovestx.co.th/products/product-user-guide/detail/manual/webtrade-connect
https://www.innovestx.co.th/products/product-user-guide/detail/manual/webtrade-thaistock
https://www.innovestx.co.th/products/product-user-guide/detail/manual/webtrade-tfex
https://www.innovestx.co.th/products/product-user-guide/detail/manual/webtrade-notification
https://www.innovestx.co.th/products/product-user-guide/detail/manual/webtrade-indicator
https://www.innovestx.co.th/products/product-user-guide/detail/manual/forgot-password-streaming-app
https://www.innovestx.co.th/products/product-user-guide/detail/manual/DA-Open-API-Manual
https://www.innovestx.co

In [13]:
manual_links = list(manual_links)
titles = []
for link in manual_links:
    response = requests.get(link,headers=headers)
    page_html = response.content
    page_soup = soup(page_html, 'html.parser')

    h1_tag = page_soup.find('h1')
    print(h1_tag)
    if h1_tag:
        title_text = h1_tag.get_text(strip=True)
        titles.append(title_text)
    else:
        titles.append('N/A')

    

<h1 class="font_medium teaser_small__text">
                    Digital Asset : การถอนเงินบาท
                </h1>
<h1 class="font_medium teaser_small__text">
                    ขั้นตอนการเปิดบัญชี TFEX, DRx และระบบซื้อขายหลักทรัพย์ต่างประเทศ (ทวีปยุโรป ทวีปเอเชีย และแคนาดา) – ผ่านแอปพลิเคชัน Streaming
                </h1>
<h1 class="font_medium teaser_small__text">
                    เติมเงินเข้าบัญชีบริหารเงินสดผ่าน QR Code
                </h1>
<h1 class="font_medium teaser_small__text">
                    วิธีการถอนเหรียญ
                </h1>
<h1 class="font_medium teaser_small__text">
                    ดูรายละเอียดพอร์ตลงทุน (Real time)
                </h1>
<h1 class="font_medium teaser_small__text">
                    ซื้อกองทุนรวม
                </h1>
<h1 class="font_medium teaser_small__text">
                    ขั้นตอนการซื้อประกัน FWD สำหรับลูกค้า InnovestX
                </h1>
<h1 class="font_medium teaser_small__text">
                    วิธีแก้เมื่อลืมอีเมล ห

## Extract Information and Save in Separate File

In [85]:
for link in manual_links:
    response = requests.get(link,headers=headers)
    page_html = response.content
    page_soup = soup(page_html, 'html.parser')
    #useful content is in section_2
    s2 = page_soup.find('section', class_='section_2')

    text_content = []
    links = []
    image_urls = []

    # Extract text content
    for i,elem in enumerate(s2.find_all(['p','h1', 'h2', 'h3', 'div','li','span'])):
        if elem.get_text(strip=True, separator=' ') == '':
            pass
        elif elem.name in ['h1', 'h2','h3','li']:
            text_content.append(elem.name+": "+elem.get_text(strip=True, separator=' '))
        elif elem.name == 'span' and 'class' in elem.attrs and ('font_main' in elem['class'] or 'font_light' in elem['class'] or 'font_medium' in elem['class']):
            text_content.append(elem.name+": "+elem.get_text(strip=True, separator=' '))
        elif 'class' in elem.attrs and 'slide_description' in elem['class']:
            text_content.append(elem.name+": "+elem.get_text(strip=True, separator=' '))
        elif 'class' in elem.attrs and 'blog_details__body_left_article_bullet_number' in elem['class']:
            text_content.append(elem.name+": "+elem.get_text(strip=True, separator=' '))
        elif 'style' in elem.attrs and 'font-size: 15px' in elem['style']:
            text_content.append(elem.name+": "+elem.get_text(strip=True, separator=' '))

    #Extract all links
    for a_tag in s2.find_all('a'):
        link = a_tag.get('href')
        if link:
            if not link.startswith('http'):
                link = 'https://www.innovestx.co.th' + link
            links.append(link)

    # Extract all image URLs
    for img_tag in s2.find_all('img'):
        img_url = img_tag.get('src')
        if img_url:
            if not img_url.startswith('http'):
                img_url = 'https://www.innovestx.co.th' + img_url.split('?')[0]
            image_urls.append(img_url)

    output_path = "/Users/bsaejia/Documents/2024_innovestx/innovestx-rag/data/txt"
    filename = "innovestx_pug_"+link.split('/')[-1]+".txt"
    with open(os.path.join(output_path,filename), 'w') as file:
        # Write text content
        file.write("Text Content:\n")
        for text in text_content:
            file.write(text + "\n")

        # Write links
        file.write("\nLinks:\n")
        for link in links:
            file.write(link + "\n")

        # Write image URLs
        file.write("\nImage URLs:\n")
        for img_url in image_urls:
            file.write(img_url + "\n")

# Print the extracted text, links, and image URLs
# print("Text Content:")
# for text in text_content:
#     print(text)

# print("\nLinks:")
# for link in links:
#     print(link)

# print("\nImage URLs:")
# for img_url in image_urls:
#     print(img_url)




## Create Dataset

In [88]:
bodies = []
titles = []
topics = []
subtopics = []
for link in manual_links:
    response = requests.get(link,headers=headers)
    page_html = response.content
    page_soup = soup(page_html, 'html.parser')
    #title
    h1_tag = page_soup.find('h1')
    #useful content is in section_2
    s2 = page_soup.find('section', class_='section_2')

    if h1_tag:
        title_text = h1_tag.get_text(strip=True)
        titles.append(title_text)
        print(title_text)
    else:
        titles.append('N/A')
        print('N/A')

    bodies.append(s2.get_text(strip=True, separator=' '))
    print(s2.get_text(strip=True, separator=' '))
    print('-----------------')
    topics.append('pug')
    subtopics.append(link.split('/')[-1])

df = pd.DataFrame(columns=['text','context','topic','subtopic'])
df['text'] = [title+"\n\n"+body for title, body in zip(titles,bodies)]
df['context'] = ['data/txt/innovestx_pug_'+subtopic+'.txt' for subtopic in subtopics]
df['topic'] = topics
df['subtopic'] = subtopics

Digital Asset : การถอนเงินบาท
การถอนเงินบาทจะมีอยู่ 2 ประเภทดังนี้ 1. ถอนเงินจากบัญชีสินทรัพย์ดิจิทัล ไปยังบัญชีบริหารเงินสด 2. ถอนเงินจากบัญชีบริหารเงินสด ไปยังบัญชีธนาคารไทยพาณิชย์ของลูกค้า 01 ถอนเงินจากบัญชีสินทรัพย์ดิจิทัล ไปยังบัญชีบริหารเงินสด 1. เลือก "สินทรัพย์ดิจิทัล" 2. เลือก "กระเป๋า" 3. เลือก "บัญชีสินทรัพย์ดิจิทัล" 4. เลือก "ถอนเงิน" 5. เลือกบัญชี 6. เลือกบัญชีบริหารเงินสด 7. ระบุจำนวนเงินที่ต้องการถอนไปยังบัญชีบริหารเงินสดโดยมีขั้นต่ำอยู่ที่ 2,000 บาท 8. ตรวจสอบข้อมูลและเลือก "ยืนยัน" 9. ใส่รหัส PIN 10. ได้รับรายการส่งคำสั่งสำเร็จ 02 การถอนเงินจากบัญชีบริหารเงินสด ไปยังบัญชีธนาคารไทยพาณิชย์ของลูกค้า 1. เลือก "บัญชีบริหารเงินสด" 2. เลือก "ถอนเงิน" 3. เลือกบัญชี 4. เลือกบัญชีที่ต้องการถอนเงิน 5. เลือกวิธีชำระเงินและระบุจำนวนเงินที่ต้องการถอน การเลือกวิธีการถอนเงินมี 2 วิธีดังนี้ 1. ถอนฟรี โดยเงินจะเข้าบัญชีของลูกค้าภายในเวลา 3 ชั่วโมงหลังจากทำรายการ 2. ถอนทันที โดยมีค่าธรรมเนียม 20 บาทต่อรายการ ขั้นต่ำในการถอนเงิน 2,000 บาท หมายเหตุ: ㆍรายการถอนเงิน ระหว่างเวลา 06.00 น. - 19

In [89]:
df

,text,context,topic,subtopic
0,Digital Asset : การถอนเงินบาท\n\nการถอนเงินบาท...,data/txt/innovestx_pug_DA-manual-01.txt,pug,DA-manual-01
1,"ขั้นตอนการเปิดบัญชี TFEX, DRx และระบบซื้อขายหล...",data/txt/innovestx_pug_Manual-TFEX.txt,pug,Manual-TFEX
2,เติมเงินเข้าบัญชีบริหารเงินสดผ่าน QR Code\n\nเ...,data/txt/innovestx_pug_topup-qrcode.txt,pug,topup-qrcode
3,วิธีการถอนเหรียญ\n\nวิธีการถอนเหรียญ 1. เข้าเว...,data/txt/innovestx_pug_DA-manual-webtrade08.txt,pug,DA-manual-webtrade08
4,ดูรายละเอียดพอร์ตลงทุน (Real time)\n\nดูรายละเ...,data/txt/innovestx_pug_Real-time.txt,pug,Real-time
...,...,...,...,...
61,วิธีการซื้อขายหุ้นไทยบน TradingView\n\nวิธีการ...,data/txt/innovestx_pug_webtrade-thaistock.txt,pug,webtrade-thaistock
62,ถอนเงินผ่านบัญชีบริหารเงินสด\n\nถอนเงินผ่านบัญ...,data/txt/innovestx_pug_ei-cashbalance-withdraw...,pug,ei-cashbalance-withdrawal
63,การซื้อขายหุ้นต่างประเทศ\n\nการซื้อหุ้นต่างประ...,data/txt/innovestx_pug_foreign-stock-trading.txt,pug,foreign-stock-trading
64,ซื้อขายหุ้นไทยผ่าน Streaming Application\n\nซื...,data/txt/innovestx_pug_Streaming-Application.txt,pug,Streaming-Application


In [91]:
df.to_pickle('/Users/bsaejia/Documents/2024_innovestx/innovestx-rag/data/innovestx_pug.pkl')